In [ ]:
import database
import importlib
import os
from datetime import datetime

from snowballing.operations import load_work, load_citations, reload
from snowballing.operations import work_by_varname, load_work_map_all_years
from snowballing.models import DB
from snowballing.jupyter_utils import idisplay, work_button, new_button
from snowballing.selenium_scholar import SeleniumScholarQuerier
from snowballing.dbmanager import set_attribute

## Check Work

In [ ]:
reload()
result = [
    idisplay(work_button(w))
    for _, w in load_work_map_all_years()
    if w._category == "work"
]
len(result)

## Check Due

In [ ]:
reload()
result = [
    idisplay(work_button(w))
    for _, w in load_work_map_all_years()
    if w._category == "unrelated"
    and not hasattr(w, "_due")
]
len(result)

## Check Title

In [ ]:
reload()
result = [
    idisplay(work_button(w))
    for _, w in load_work_map_all_years()
    if not hasattr(w, "title")
    or w.title.endswith(".")
]
len(result)

## Check Author

In [ ]:
reload()
result = [
    idisplay(work_button(w))
    for _, w in load_work_map_all_years()
    if not hasattr(w, "author")
    or w.author.endswith(".")
    and not w.author.endswith(" al.")
    and not w.author[-3] in "- " and w.author[-2].isalpha()
]
len(result)

## Check File/Link

In [ ]:
def check_file(arg):
    k, w = arg
    if hasattr(w, "_url") or w._category in ("nofile", "site"):
        return (k, w, "")
    if isinstance(getattr(w, "_file"), str):
        if os.path.exists(os.path.join("files", w._file.split("#")[0])):
            return (k, w, "")
        return (k, w, "inexistent")
    if w.file is None:
        return (k, w, "no attribute")
    return (k, w, "type error")

reload()
result = [
    idisplay(work_button(w), r)
    for _, w, r in map(check_file, load_work_map_all_years())
    if r
]
len(result)

In [ ]:
declared_files = {
    w._file.split("#")[0] for k, w in load_work_map_all_years()
    if isinstance(w._file, str)
}
set(os.listdir('files')) - declared_files - {'alternatives'}

## Check nofile

In [ ]:
from IPython.display import display
reload()
result = [
    idisplay(work_button(w))
    for _, w in load_work_map_all_years()
    if w._category == "nofile"
]
len(result)

## Scholar

In [ ]:
querier = None
reload()
worklist = sorted(
    [k for k, w in load_work_map_all_years() if not getattr(w, "_scholar_ok", False)],
    key=lambda x: (int(x[-5:-1]), x)
)
len(worklist)

In [ ]:
if worklist and querier is None:
    querier = SeleniumScholarQuerier()
    querier.apply_settings(10, 4)

In [ ]:
from snowballing.snowballing import ScholarUpdate
supdate = ScholarUpdate(querier, worklist, force=False)

In [ ]:
supdate

In [ ]:
{w._scholar_ok for k, w in load_work_map_all_years() if getattr(w, "_scholar_ok", False)}

## Forward

Finding scholar cluster: http://webapps.stackexchange.com/questions/45333/how-to-create-a-citation-alert-for-a-paper-without-citation-in-google-scholar

In [ ]:
def set_snowball_date_button(key, date):
    def click(w):
        set_attribute(key, "snowball", "datetime({0.year}, {0.month}, {0.day})".format(date))
    return new_button("Set date", click)
    
date = datetime(2017, 3, 6)
reload()
result = [
    idisplay(
        work_button(w),
        set_snowball_date_button(k, date),
        '"{}", citation_file="{}"'.format(k, getattr(w, "_citation_file", w._metakey))
            if hasattr(w, '_scholar') else w.title,
        label=False
    )
    for k, w in load_work_map_all_years()
    if hasattr(w, '_snowball')
    and w._category == "snowball"
    and w._snowball != date
]
len(result)

## Tracking

In [ ]:
def set_tracking_button(key):
    def click(w):
        set_attribute(key, "_tracking", "alert")
    return new_button("Set track", click)

reload()
result = [
    idisplay(
        work_button(w),
        set_tracking_button(k),
        "https://scholar.google.com/scholar?cluster={}&hl=en&as_sdt=2005&sciodt=0,5".format(w._cluster_id)
            if hasattr(w, '_cluster_id') else w.title,
        getattr(w, '_scholar', ''),
        label=False,
    )
    for k, w in load_work_map_all_years()
    if getattr(w, "_tracking", "") not in ("alert", "impossible")
    and w._category == "snowball"
]
len(result)

In [ ]:
len({k for k, w in load_work_map_all_years() if w._category == "snowball"})

## Dash

In [ ]:
def check_dash(ele):
    k, w = ele
    def create_button(attr):
        def click(_):
            set_attribute(k, attr, str(getattr(w, attr)).replace("-", "--"))
        return new_button("Set " + attr, click)
    reasons = []
    if isinstance(getattr(w, 'pages', ''), int):
        reasons.append(create_button('pages'))
    elif getattr(w, 'pages', '').count('-') not in (0, 2):
        reasons.append(create_button('pages'))
    if isinstance(getattr(w, 'volume', ''), int):
        reasons.append(create_button('volume'))
    elif getattr(w, 'volume', '').count('-') not in (0, 2):
        reasons.append(create_button('volume'))
    if isinstance(getattr(w, 'number', ''), int):
        reasons.append(create_button('number'))
    elif getattr(w, 'number', '').count('-') not in (0, 2):
        reasons.append(create_button('number'))
    return k, w, reasons

reload()
result = [
    idisplay(work_button(w), *r)
    for k, w, r in map(check_dash, load_work_map_all_years())
    if r
]
len(result)